# Задание по теме «Коллаборативная фильтрация»

#### ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

https://russianblogs.com/article/8373229539/ - инфо с методами библиотеки surprise

## Решение

In [1]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy # всевозможные метрики
from surprise import Reader # класс считывания данных
from surprise.model_selection import train_test_split

import pandas as pd

In [2]:
#User information (users.dat table, data without entry information)
unames = ['user_id','gender','age','occupation','zip']
users = pd.read_table('users.dat', sep='::', header=None, names=unames, engine='python')

#Rating information
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=rnames, engine='python')

# #Movie information
mnames = ['movie_id','title','genres']
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='ISO-8859-1')

In [3]:
data=pd.merge(pd.merge(ratings,users),movies)

In [4]:
data.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   user_id     1000209 non-null  int64 
 1   movie_id    1000209 non-null  int64 
 2   rating      1000209 non-null  int64 
 3   timestamp   1000209 non-null  int64 
 4   gender      1000209 non-null  object
 5   age         1000209 non-null  int64 
 6   occupation  1000209 non-null  int64 
 7   zip         1000209 non-null  object
 8   title       1000209 non-null  object
 9   genres      1000209 non-null  object
dtypes: int64(6), object(4)
memory usage: 83.9+ MB


In [6]:
dataset = pd.DataFrame({ # surprise нужен определённый формат данных 
    'uid': data.user_id,# первый столбец id пользователя
    'iid': data.movie_id, # второй столбец id фильма или нзавания
    'rating': data.rating # третий столбец рейтинг пользователя
})

In [7]:
# это нужно для преобразователя, чтобы понять какой минимум есть и какой максимум
ratings.rating.min()

1

In [8]:
ratings.rating.max()

5

In [9]:
reader = Reader(rating_scale=(1.0, 5.0))
dt = Dataset.load_from_df(dataset, reader) # формат данных разреженая матрица - наны не весомы, а остаьлные значения хранятся для экономии памяти

In [10]:
trainset, testset = train_test_split(dt, test_size=.15)

#####  Первая модель user_based KNNWithMeans

In [11]:
algo = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [12]:
test_pred = algo.test(testset)

In [13]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8864


0.8864428590904863

In [14]:
algo.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=3.580829639004584, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [15]:
from surprise.model_selection.validation import cross_validate

In [16]:
cross_validate(algo, dt, measures=[u'rmse'], cv=5, n_jobs=1)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.89029487, 0.89024828, 0.89115506, 0.88896226, 0.89010409]),
 'fit_time': (112.8629641532898,
  111.53045439720154,
  110.2348780632019,
  113.1735212802887,
  111.76449680328369),
 'test_time': (140.5456395149231,
  141.45300006866455,
  145.94227766990662,
  158.8815941810608,
  152.49366688728333)}

#####   Вторая модель unuser_based KNNWithMeans

In [17]:
algo_2 = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [18]:
cross_validate(algo_2, dt, measures=[u'rmse'], cv=5, n_jobs=1)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.86093952, 0.86387177, 0.86318279, 0.86615316, 0.86365177]),
 'fit_time': (54.76084899902344,
  58.23543357849121,
  55.696778535842896,
  55.95234751701355,
  55.49998712539673),
 'test_time': (93.21248269081116,
  99.44039225578308,
  95.21621918678284,
  92.82108783721924,
  94.65982604026794)}

##### Третья модель SVD - скрытые состояния

In [19]:
from surprise import SVD

In [27]:
algo_3 = SVD(n_factors=25, n_epochs=20)

In [28]:
cross_validate(algo_3, dt, measures=[u'rmse'], cv=5)

{'test_rmse': array([0.86946345, 0.86927837, 0.87220338, 0.86923309, 0.87263452]),
 'fit_time': (22.994571447372437,
  24.658490419387817,
  23.931807041168213,
  23.289746046066284,
  23.074722290039062),
 'test_time': (2.2935092449188232,
  2.9087226390838623,
  2.2451679706573486,
  2.8992300033569336,
  2.0351531505584717)}